## invoice parser

In [ ]:
#!pip install pymupdf pdfplumber requests --quiet
## using bootcamp3 python3.11.4 on Gunnars book

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [16]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

display(prompt)

FewShotPromptTemplate(input_variables=['input'], examples=[{'question': 'Who lived longer, Muhammad Ali or Alan Turing?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n'}, {'question': 'When was the founder of craigslist born?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n'}, {'question': 'Who was the maternal grandfather of George Washington?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washi

## Now we have a FewShotPromptTemplate
Use the prompt template it by invoking it with a question labeled "input" in this case. the input will need to match the examples and should therefore generatate simialar output

These templates can do other things for example if you want shorter or better targeted examples from the template examples you can use a vector similarity search to pull only the most similar examples see ref 2 for details

We can see that the template object is convetnient but not extrordinary. it just emits structured text to be passed into the LLM prompt 

In [17]:
print(prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string())

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

In [18]:
#import os
import pandas as pd
import requests
import math

import acd_datatool as acd

verbose = True
pd.options.display.max_columns = None

#######################################################
import acd_local_llm as locllm

#######################################################
import os
import json

# # Function to extract text from PDF
# def extract_text_from_pdf(pdf_path):
#     doc = fitz.open(pdf_path)
#     text = "\n".join(page.get_text("text") for page in doc)
#     return text

In [22]:

def llm_inv_parse(extracted_data, prompt_input, conect_str="http://localhost:11434/api/generate"):

    prompt=f"""
    {prompt_input}

    Text:
    {extracted_data}

    Return a JSON response.
    """
    # Send extracted text to Ollama
    payload = {
        "model": "mistral",  # Change model as needed
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(conect_str, json=payload)

    # Parse response
    if response.status_code == 200:
        data = response.json()
        print("Parsed Invoice Data:", data["response"])
    else:
        print(f"Error: {response.status_code}, {response.text}")

    return data

def process_pdf_invoices(folder_path, lama_endpoint, method='llama', verbose=False):
    """
    Processes all PDF invoices in a folder using the LaMA 3.1 API to extract invoice and line-item data.
    ##############################
    method='llama' # parses data with llama 3.1 endpoint
    method='report' # prints existing files 
    method='chk_report' # TODO, make checkilist and check job and invoice sheet for the item 
    """
    invoices_data = []
    jobs_data = []



    for file_name in os.listdir(folder_path):
        #print(f'working on filename{file_name}')
        allextracted_data = []

        if file_name.endswith('.pdf'):  # Only process PDF files

            file_path = os.path.join(folder_path, file_name)
            print(f'processing {file_path}')
            if method=='llama':
                print(f'Using {method} method')
                try:
                    #Extract data from INVOICE, with pdf parser or LLM
                    #extracted_data = parse_pdf_with_lama(file_path, lama_endpoint)
                    # get the text from the invoice PDF
                    extracted_data = acd.extract_text_locally(file_path)
                    # Collect the extracted data
                    allextracted_data.append(extracted_data)
                    
                    prompt_inv = f"""
                    Extract the following details from the invoice text:
                    - 'Worker Name'
                    - 'Date Submitted'
                    - 'Total Amount Invoiced'
                    """

                    prompt_jobs = f"""
                    Extract the following details from the invoice text:
                    - "List_of_Job_dictionaries" 
                    The "list of job dictionaries" should have the following keys for details for each item:  
                    {"Job ID",  "Units Worked", "Rate", "Amount Invoiced", "Item Name", "Job item name"}
                    if a detail is not present, return 'Unknown' or 'na' as appropriate."""


                    # - Units Worked for each job
                    # - Amount Invoiced for each job
                    # - Rate for each job
                    # - Job item name

                    #parse the invoice response
                    parsed_inv_data=llm_inv_parse(extracted_data, prompt_inv, conect_str="http://localhost:11434/api/generate")
                    response_dict = json.loads(parsed_inv_data["response"])    

                    #parse the jobs response
                    parsed_job_data=llm_inv_parse(extracted_data, prompt_jobs, conect_str="http://localhost:11434/api/generate")
                    responsejob_dict = json.loads(parsed_job_data["response"])    
                    
                    #add metadata to file structure common to all files
                    
                    worker_name = response_dict.get('Worker Name', 'Unknown')
                    submission_date = response_dict.get('Date Submitted', 'Unknown')
                    #file_name = response_dict.get('file_name', 'Unknown')
                    #file_path = response_dict.get('file_path', 'Unknown')

                    #append data to invoices_data sheet
                    invoices_data.append({
                        'invoice': file_name,
                        'File Path': file_path,
                        'Submission Date': submission_date,
                        'year': submission_date,	
                        'Inv_paid_date': 'NA',	
                        'inv_paid_link': 'NA',	
                        'job_start': response_dict.get('X', 'Unknown'),	
                        'Job_end': response_dict.get('X', 'Unknown'),
                        'to_client': response_dict.get('X', 'Unknown'),	
                        'broker_chain': response_dict.get('X', 'Unknown'),	
                        'inv_from': worker_name,	
                        'end_client': response_dict.get('X', 'Unknown'),	
                        'job_name': response_dict.get('List of Jobs', 'Unknown'),	
                        'task_descr': response_dict.get('X', 'Unknown'),
                        'worker': worker_name,
                        'status': 'submitted',
                        'inv_dollars': response_dict.get('Total Amount Invoiced', None)
                    })

                    # append data to jobs_data sheet

                    
                    for item in responsejob_dict.get('List_of_Job_dictionaries'):
                        jobs_data.append({
                            'Teacher': worker_name,
                            'start_date': item.get('x', 'Unknown'),	
                            'end_date': item.get('x', 'Unknown'),	
                            'job': item.get("Job ID", 'Unknown'),
                            'Task_Descr': item.get("Job item name", 'Unknown'),		
                            'type': item.get("Item Name", 'Unknown'),		
                            'ACD_bill_dt': item.get('x', 'Unknown'),		
                            'ACD_pay_dt': item.get('x', 'Unknown'),		
                            'ACD_inv_link': item.get('x', 'Unknown'),	
                            'teacher_pay_dt': item.get('x', 'Unknown'),	
                            'ACD_inv_status': item.get('x', 'Unknown'),	
                            'ACD_invoice': item.get('x', 'Unknown'),
                            'ACD_inv_link': item.get('x', 'Unknown'),	
                            'AttributeError': item.get('x', 'Unknown'),
                            'Worker_invoice':file_name,
                            'worker_inv_link':file_path,
                            'Inv_line_item': item.get('x', 'Unknown'),
                            'direct_client': item.get('x', 'Unknown'),
                            'End_client': item.get('x', 'Unknown'),
                            'project': item.get("Job ID", 'Unknown'),
                            'teacher_pay_amt': item.get('x', 'Unknown'),
                            'worker_item_rate': item.get("Rate", None),
                            'days': item.get('x', 'Unknown'),
                            'item_quantity': item.get("Units Worked", 'na'),
                            'billed_item_total': item.get("Amount Invoiced", 'na'),
                            'ACD_Item_Rate': item.get('x', 'Unknown'),
                            'ACD_overhead_Prc': item.get('x', 'Unknown'),
                            'ACD_day_rate': item.get('x', 'Unknown'),
                            'notes': item.get('x', 'Unknown'),
                            })
                

                    if verbose:
                        print(f'##'*15)    
                        print(f'Extracted this data from the pdf {file_name}')

                        print(f'##'*15)
                        print(f'\n\n I got this back from the LLM inv {response_dict}')
                        print(f'##'*15)
                        print(f'\n\n I got this back from the LLM jobs {responsejob_dict}')
                        print(f'##'*15)
                    else:
                        print('I extracted data from the pdf {file_name} and parsed it with the LLM')
                    
                
                except Exception as e:
                    print(f"Error processing file {file_name}: {e}")
            elif method=='report':
                print(file_name)
            elif method=='check_report':
                print(file_name)
            else:    
                print('specify a valid method')

    invoices_df = pd.DataFrame(invoices_data)
    jobs_df = pd.DataFrame(jobs_data)

    if verbose:
        print(f'invoices_data: {invoices_data}/n/n')
        print(f'jobs_data: {jobs_data}/n/n')
        display(invoices_df.head())
        display(jobs_df.head())

    # make JSON‑safe by replacing NaN with None
    invoices_df = invoices_df.where(pd.notnull(invoices_df), None)
    jobs_df     = jobs_df.where(pd.notnull(jobs_df),     None)

    
    return invoices_df, jobs_df, allextracted_data

def check_for_invalid_floats(data):
    
  """ 
  check data frame or json for invalid floats

  usage: inspect 'bad[3]' to get the full data, run code to get report

  bad = check_for_invalid_floats(jobs_df)
  if bad:
      print("Found invalid float:", bad[1], "in", bad[0])
  """
  # if a DataFrame is passed in, convert to nested dict
  if hasattr(data, "to_dict"):
      data = data.to_dict()         
  
  for key, val in data.items():
      if isinstance(val, dict):
          inner = check_for_invalid_floats(val)
          if inner:
              # prepend parent key
              return f" Col:'{key}', Row:'{inner[0]}'", inner[1], data
      else:
          if isinstance(val, float) and not math.isfinite(val):
              return key, val, data
  return None

    


##Check that the LLM is running

In [24]:
connection_str="http://localhost:11434/api/generate"
lama_endpoint = "http://localhost:11434"

locllm.check_endpoint_status(lama_endpoint)

locllm.test_llm("What is the capital of kathmandu?", connection_str)


Error connecting to LaMA API: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [11]:
if __name__ == "__main__":
    base64mode=False

    ############### API credential loading from local folder  ########
    #### secrets_ls_file1: is the financial tracker service acct
    #### secrets_ls_file2: are the kanban boards individual APIs

    verbose=False
    #method=input("which connection method should I use (local or colab_auth)?") 
    method = 'L'

    g_conect, api_dict=acd.load_method(method, secrets_ls=
            ['/Users/gunnarkleemann/.ssh/financetrack-acd-aa2d12d1f346.json',
            '/Users/gunnarkleemann/.ssh/acd-api.json'])

    #/Users/gunnarkleemann/.ssh/acd-internal-analytics-375db6d96d79.json

    api_token_edu_board=api_dict['api_token_edu_board']
    api_token_contr_board=api_dict['api_token_contract_board']
    #folder_path = '/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_financial/Invoices/unfinished/FromSvcAcct_Incoming'
    folder_path = "/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_financial/Invoices/unfinished/FromSvcAcct_Incoming"  # Folder with PDF invoices
    lama_endpoint = "http://localhost:11434/api/process"#/process"  # LaMA 3.1 API endpoint
    #"/Users/gunnarkleemann/.ssh/acd-internal-analytics-375db6d96d79.json"
    credentials_file = '/Users/gunnarkleemann/.ssh/financetrack-acd-aa2d12d1f346.json' # Google service account key
    share_with_emails = ["gunnar@austincapitaldata.com", "gunnarkl@gmail.com"]  # Emails to share the sheets with
    sender_email = "service@austincapitaldata.com"  # Sender email for notifications
    sender_password = api_dict['service_acct_email_psswd']  # Sender email password (use app-specific passwords if needed)- this can't be your email login, go to 'Google App Passwords'


In [12]:
# Process invoices
#invoices_df, jobs_df = locllm.process_pdf_invoices(folder_path, lama_endpoint, method='llama')
invoices_df, jobs_df, extracted_data = process_pdf_invoices(folder_path, lama_endpoint, method='llama', verbose=True)

print(f"REPORTING: invioces URL {invoices_df.head()} jobs URL {jobs_df.head()}")

dfs = {
    "invoices_df": invoices_df,
    "jobs_df":      jobs_df,
}


for i in dfs.items():
    bad = check_for_invalid_floats(i[1])
    if bad:
        print(f"Found invalid float in: {bad[0]} value: {bad[1]}")
    else :
        print(f"No invalid floats found in {i[0]}")

### prepare dfs for json parsing
## change nan to none...
jobs_df = jobs_df.fillna(value='None')
invoices_df = invoices_df.fillna(value='None')

# Create Google Sheets for intermediate validation
invoices_sheet_url = acd.create_google_sheet("Invoices Review Sheet", invoices_df, credentials_file, share_with_emails)
jobs_sheet_url = acd.create_google_sheet("Jobs Review Sheet", jobs_df, credentials_file, share_with_emails)

print(f"REPORTING: invioces URL {invoices_sheet_url} jobs URL {jobs_sheet_url}")

# Send email notifications
acd.send_email_notification(
    sender_email,
    sender_password,
    share_with_emails,
    {"invoices": invoices_sheet_url, "jobs": jobs_sheet_url}
)

print("Google Sheets created and shared successfully. Email notifications sent.")

processing /Users/gunnarkleemann/Dropbox/ACD_folders/ACD_financial/Invoices/unfinished/FromSvcAcct_Incoming/Invoice_Kevin_Martin_2025-08-cdc.pdf
Using llama method
Parsed Invoice Data:  {
      "Worker Name": "Kevin Martin",
      "Date Submitted": "2025/08/29",
      "Total Amount Invoiced": "$4,400.00"
   }
Parsed Invoice Data:  {
      "List_of_Job_dictionaries": [
        {
          "Job ID": "Gunnar Kleemann Kevin Martin 2025-08-cdc",
          "Units Worked": "40.00",
          "Rate": "$110.00",
          "Amount Invoiced": "$4,400.00",
          "Item Name": "Lead Instructor - CDC Part 4 - Shiny Apps",
          "Job item name": "Unknown" // since there is no specific job item name mentioned in the text
        }
      ]
    }
Error processing file Invoice_Kevin_Martin_2025-08-cdc.pdf: Expecting ',' delimiter: line 9 column 38 (char 324)
processing /Users/gunnarkleemann/Dropbox/ACD_folders/ACD_financial/Invoices/unfinished/FromSvcAcct_Incoming/Christine Sako Invoice #001.pdf
U

,invoice,File Path,Submission Date,year,Inv_paid_date,inv_paid_link,job_start,Job_end,to_client,broker_chain,inv_from,end_client,job_name,task_descr,worker,status,inv_dollars
0,Christine Sako Invoice #001.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,05/21/2025,05/21/2025,NA,NA,Unknown,Unknown,Unknown,Unknown,Christine Sako,Unknown,Unknown,Unknown,Christine Sako,submitted,"$1,275.00"
1,Invoice_Kevin_Martin_2025-09-nvidia.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,2025/08/29,2025/08/29,NA,NA,Unknown,Unknown,Unknown,Unknown,Kevin Martin,Unknown,Unknown,Unknown,Kevin Martin,submitted,$800.00


,Teacher,start_date,end_date,job,Task_Descr,type,ACD_bill_dt,ACD_pay_dt,ACD_inv_link,teacher_pay_dt,ACD_inv_status,ACD_invoice,AttributeError,Worker_invoice,worker_inv_link,Inv_line_item,direct_client,End_client,project,teacher_pay_amt,worker_item_rate,days,item_quantity,billed_item_total,ACD_Item_Rate,ACD_overhead_Prc,ACD_day_rate,notes
0,Christine Sako,Unknown,Unknown,Gunnar Kleemann Christine Sako 001,Unknown,Custom Applied Data Science with Python (WA3109),Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Christine Sako Invoice #001.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,Gunnar Kleemann Christine Sako 001,Unknown,$75.00,Unknown,16,"$1,200.00",Unknown,Unknown,Unknown,Unknown
1,Kevin Martin,Unknown,Unknown,TA - NVIDIA,NVIDIA Building Transformer Based NLP Apps,Xformer Based NLP Apps,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Invoice_Kevin_Martin_2025-09-nvidia.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,TA - NVIDIA,Unknown,$100.00,Unknown,8.00,$800.00,Unknown,Unknown,Unknown,Unknown


REPORTING: invioces URL                                    invoice  \
0          Christine Sako Invoice #001.pdf   
1  Invoice_Kevin_Martin_2025-09-nvidia.pdf   

                                           File Path Submission Date  \
0  /Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...      05/21/2025   
1  /Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...      2025/08/29   

         year Inv_paid_date inv_paid_link job_start  Job_end to_client  \
0  05/21/2025            NA            NA   Unknown  Unknown   Unknown   
1  2025/08/29            NA            NA   Unknown  Unknown   Unknown   

  broker_chain        inv_from end_client job_name task_descr          worker  \
0      Unknown  Christine Sako    Unknown  Unknown    Unknown  Christine Sako   
1      Unknown    Kevin Martin    Unknown  Unknown    Unknown    Kevin Martin   

      status inv_dollars  
0  submitted   $1,275.00  
1  submitted     $800.00   jobs URL           Teacher start_date end_date                          

In [41]:
jobs_df


,Teacher,start_date,end_date,job,Task_Descr,type,ACD_bill_dt,ACD_pay_dt,ACD_inv_link,teacher_pay_dt,ACD_inv_status,ACD_invoice,AttributeError,Worker_invoice,worker_inv_link,Inv_line_item,direct_client,End_client,project,teacher_pay_amt,worker_item_rate,days,item_quantity,billed_item_total,ACD_Item_Rate,ACD_overhead_Prc,ACD_day_rate,notes
0,Kirti Rohit Kamerkar,Unknown,Unknown,1,Classical object detection,Computer Vision,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,KirtiJuly_2025.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,1,Unknown,$35 per hour,Unknown,17,$6075,Unknown,Unknown,Unknown,Unknown
1,Kirti Rohit Kamerkar,Unknown,Unknown,2,Unknown,Data society TA,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,KirtiJuly_2025.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,2,Unknown,$75 per hour,Unknown,10,$750,Unknown,Unknown,Unknown,Unknown
2,Kirti Rohit Kamerkar,Unknown,Unknown,3,Unknown,NVIDIA TA,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,KirtiJuly_2025.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,3,Unknown,$75 per hour,Unknown,8,$600,Unknown,Unknown,Unknown,Unknown
3,Kirti Rohit Kamerkar,Unknown,Unknown,4,Unknown,Data Society dev,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,KirtiJuly_2025.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,4,Unknown,$55 per hour,Unknown,34,$1870,Unknown,Unknown,Unknown,Unknown
4,Kirti Rohit Kamerkar,Unknown,Unknown,5,na,Data society study,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,KirtiJuly_2025.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,5,Unknown,na,Unknown,Unknown,na,Unknown,Unknown,Unknown,Unknown
5,Kevin Martin,Unknown,Unknown,Gunnar Kleemann Kevin Martin 2025-08-McK-Boot,Unknown,McKinsey GenAI Bootcamp,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,KMartin_2025-08-McK-Boot.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,Gunnar Kleemann Kevin Martin 2025-08-McK-Boot,Unknown,$110.00,Unknown,24.00,"$2,640.00",Unknown,Unknown,Unknown,Unknown
6,Christine Sako,Unknown,Unknown,Gunnar Kleemann Christine Sako 001,Unknown,Custom Applied Data Science with Python (WA3109),Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Christine Sako Invoice #001.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,Gunnar Kleemann Christine Sako 001,Unknown,$75.00,Unknown,16,"$1,200.00",Unknown,Unknown,Unknown,Unknown
7,Kevin Martin,Unknown,Unknown,Ascendient - DS 2201 Development,Unknown,Development,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,KMartin_2025-06-All.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,Ascendient - DS 2201 Development,Unknown,$50.00,Unknown,46.49,"$2,324.50",Unknown,Unknown,Unknown,Unknown
8,Kevin Martin,Unknown,Unknown,Ascendient - DS 1101 Development,Unknown,Development,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,KMartin_2025-06-All.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,Ascendient - DS 1101 Development,Unknown,$50.00,Unknown,43.37,"$2,168.50",Unknown,Unknown,Unknown,Unknown
9,Kevin Martin,Unknown,Unknown,Ascendient - GAI 1101 Debugging,Unknown,Debugging,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,KMartin_2025-06-All.pdf,/Users/gunnarkleemann/Dropbox/ACD_folders/ACD_...,Unknown,Unknown,Unknown,Ascendient - GAI 1101 Debugging,Unknown,$50.00,Unknown,8.10,$405.00,Unknown,Unknown,Unknown,Unknown


In [42]:
jobs_df = jobs_df.fillna(value='None')
invoices_df = invoices_df.fillna(value='None')

# Create Google Sheets for intermediate validation
invoices_sheet_url = acd.create_google_sheet("Invoices Review Sheet", invoices_df, credentials_file, share_with_emails)
jobs_sheet_url = acd.create_google_sheet("Jobs Review Sheet", jobs_df, credentials_file, share_with_emails)


print(f"REPORTING: invioces URL {invoices_sheet_url} jobs URL {jobs_sheet_url}")

# Send email notifications
acd.send_email_notification(
    sender_email,
    sender_password,
    share_with_emails,
    {"invoices": invoices_sheet_url, "jobs": jobs_sheet_url}
)

print("Google Sheets created and shared successfully. Email notifications sent.")

REPORTING: invioces URL https://docs.google.com/spreadsheets/d/1XCZqTJxaTOb7OMy8_5J7LW8xhZByCfP6Bvso6tkeJJg jobs URL https://docs.google.com/spreadsheets/d/1q9Iny0X4qNr1t9noSK_sEJGycjSRJPshIwBghREIADg
Google Sheets created and shared successfully. Email notifications sent.


In [43]:
dfs = {
    "invoices_df": invoices_df,
    "jobs_df":      jobs_df,
}


In [44]:
# Create Google Sheets for intermediate validation
invoices_sheet_url = acd.create_google_sheet("Invoices Review Sheet", invoices_df, credentials_file, share_with_emails)
jobs_sheet_url = acd.create_google_sheet("Jobs Review Sheet", jobs_df, credentials_file, share_with_emails)

print(f"REPORTING: invioces URL {invoices_sheet_url} jobs URL {jobs_sheet_url}")

# Send email notifications
acd.send_email_notification(
    sender_email,
    sender_password,
    share_with_emails,
    {"invoices": invoices_sheet_url, "jobs": jobs_sheet_url}
)

print("Google Sheets created and shared successfully. Email notifications sent.")

REPORTING: invioces URL https://docs.google.com/spreadsheets/d/1q16XXCedVwgNCwTwtTNzCvs_lU7zUTELuThDcqsMcGY jobs URL https://docs.google.com/spreadsheets/d/1dpGh9X42ZzScOCmGEzuYUqgkMKznN0err5mApaMM0-Y
Google Sheets created and shared successfully. Email notifications sent.


## References

# starting point chatGPT 11/15/25
https://chatgpt.com/share/67376831-924c-800f-a6b9-83ec843514e6
    
OpenAI. (2024). ChatGPT [Large language model]. https://chatgpt.com

### templates
https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/how_to/few_shot_examples.ipynb#scrollTo=4e70bce2